# Overscan Subtraction

*J. Runnoe* <br>
*October, 2023*

This notebook will take you through the steps to overscan and bias subtract your science frames. Many of these tasks are reflected in the [astropy CCD reduction and photometry guide](https://github.com/astropy/ccd-reduction-and-photometry-guide/tree/main), so reading those notebooks can be very helpful.

---
## Contents
* [Setup](#setup)
    * [Download CCD Photometry Guide](#ccdphot)
* [Exercises](#exercises)
* [Summary](#summary)

## Setup <a class="anchor" id="setup"></a>

### Download CCD Photometry Guide <a class="anchor" id="ccdphot"></a>

I have mentioned several times that the CCD reduction and photometry guide is a useful tool. If you haven't already, I recommend downloading a local copy from Github so that you can interact with the notebooks and use some of their convenience functions.

To do this, start by creating a home for the guide on your local machine. I keep a `Software/` directory on my computer, with subdirectories for code in different languages. Therefore, I can just stick this guide in my existing `py` directory. <br>
`$ cd ~/Software/py/` <br>
`$ mkdir ccd-reduction-and-photometry-guide/` <br>

Then, use `git clone` to copy the guide to your local machine. This makes it easy to download updates and changes anytime `astropy` publishes them. <br>
`$ cd ccd-reduction-and-photometry-guide` <br>
`$ git clone https://github.com/astropy/ccd-reduction-and-photometry-guide/ .` <br>

It may take a minute to donwload everything.

---
## Exercises <a class="anchor" id="exercises"></a>


In [1]:
# import block
import numpy as np
from astropy.io import fits
from matplotlib import pyplot as plt
from matplotlib import rc
%matplotlib inline
from astropy.visualization import hist
from ccdproc import ImageFileCollection
import ccdproc as ccdp
from astropy.modeling import fitting
from astropy.modeling.models import Polynomial1D,Chebyshev1D,Legendre1D,Hermite1D
from astropy.nddata import CCDData

In [2]:
# import convenience plotting functions downloaded from 
# here: https://github.com/mwcraig/ccd-reduction-and-photometry-guide
phot_tutorial_dir = '/Users/runnojc1/Software/py/ccd-reduction-and-photometry-guide/notebooks/'
import sys
sys.path.insert(0,phot_tutorial_dir)
from convenience_functions import show_image

In [3]:
# for imaging, I like the photometry notebook plot defaults
# so use their custom style for larger fonts and figures
plt.style.use('/Users/runnojc1/Software/py/ccd-reduction-and-photometry-guide/notebooks/guide.mplstyle')

# set a couple more default parameters for the plots below
rc('font', size=20)
rc('axes', grid=True)

#### 1. Create a working directory

You should preserve a clean copy of the `Imaging/` folder that you downloaded. That is, do not work in that folder or modify those files directly. Instead, you should create a `reduced/` directory to conduct your work and copy over files as necessary there.

#### 2. Open a bias frame in Python

There are several ways to interact with images, which are outlined in [this notebook](https://github.com/astropy/ccd-reduction-and-photometry-guide/blob/main/notebooks/01-11-reading-images.ipynb). You have seen the `astropy.io.fits` approach before, but this mentions two new options. One is to use the [`CCDData`](https://ccdproc.readthedocs.io/en/latest/ccddata.html) image class, and another (useful when you have many frames) is the [`ImageFileCollection`](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.ImageFileCollection.html). 

In [4]:
data_dir = '/Users/runnojc1/Dropbox/Research/teaching/F2021/ASTR8060/Imaging/'            # raw data directory

#### 3. Inspect the header
If the BIASSEC, TRIMSEC, FILTER, and IMAGETYP header keywords are not present, [add them](https://docs.astropy.org/en/stable/io/fits/usage/headers.html). HERE, BIASSEC refers to the indices needed to select the overscan region (one of them is sufficient), and TRIMSEC refers to the region you want to extract when you trim the overscan off. Then save the file with new header keywords to `reduced/`. Open the file as an image collection and check that the header keywords are there.

You can access header keywords with code like this:

```python
hdu = fits.open('filename')
hdr = hdu[extension].header
key = hdr['KEYWORD']
```

Here, `filename` is the file you want to read in, `extension` is a number indicating the extension of the fits file you want to access the header for, and `KEYWORD` is the keyword name you want to extract the header value for. If the keyword isn't present in the header, you'll get an error. You can create new keywords and assign values to them like this:

```python
hdr['NEW_KEYWORD'] = 5
hdr['ANOTHER_KEY'] = '[0:10,20:30]'
```

A word on indexing. Python starts at 0 and goes [row,col], FITS starts at 1 and goes [col,row]. Also, Python will take a range as [low,high), where high is not included, so the upper limits will be the same for Python and FITS. The `CCDPROC` `subtract_overscan` function uses both of these, depending on how you pass it, so be careful. 

### 4. Model the overscan region

We need to perform a bias subtraction on the images. The best way to do this would be to subtract a value from each pixel, but we don't know the correct value. The overscan region doesn't capture all the pixel-to-pixel variations, and the bias frames have noise in them. Our best approach will perform the bias subtraction in two steps. First, we will subtract the overscan region and trim it off to reveal the science section (referred to as the TRIMSEC). Then, we will remove any two dimensional patterns that remain.

When we subtract the bias level indicated in the overscan region, we can do a better job and subtract a different value for each row. Figure out waht value to use by fitting a function to the pixel values in the overscan region as a function of row number. 

Determine whether it is better to use a mean or median to collapse the columns, and what function and order to fit along each row.

For model fitting, `astropy` has some handy packages for model fitting. I recommend the [astropy models](https://docs.astropy.org/en/stable/modeling/) for this model fitting and [these examples](https://learn.astropy.org/tutorials/Models-Quick-Fit.html) to learn how to 1D model fitting. 

In [4]:
# mean or median to combine columns?


In [5]:
# model overscan

#### 5. Bias subtract and trim your image
While we can do many data reduction steps by writing our own code, it's not always time efficient to reinvent the wheel. [`CCDPROC`](https://ccdproc.readthedocs.io/en/latest/index.html) is a code with functionality for astronomical CCD data reduction.

Use [`ccdproc.subtract_overscan`](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.subtract_overscan.html) and [`ccdproc.trim_image`](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.trim_image.html) with the best parameters from \#4 to subtract and trim your bias image. Display the raw bias and overscan subtracted and trimmed version in Python.

During the course of data reduction, it is common to give your variables and filenames flags to indicate which corrections have been applied. These are relatively standard:

-o overscan subtracted <br>
-t trimmed <br>
-z bias corrected <br>
-f flat field corrected <br>
-d dark corrected <br>

So for a raw frame `a093.fits`, it would be called `a093_otz.fits` after overscan, trim, and bias correction.

---
## Summary <a class="anchor" id="summary"></a>

At this point, all of you should have:
* Cloned a local copy of the CCD photomoetry guide.
* Modified the header keywords of a bias frame.
* Determined whether it is better to mean or median combine your biases.
* Modeled the overscan region.
* Overscan subtracted and trimmed a bias frame.